In [12]:
import pandas as pd
import tkinter as tk
from pyparsing import nestedExpr
from tkinter import scrolledtext




def implication(a,b):
    if a == 1 and b == 0:
        return 0
    else:
        return 1
    
negate = lambda a: not a
log_and = lambda a,b: a and b
log_or = lambda a,b: a or b
log_equal = lambda a,b: int(a==b)

def generate_next_list(amount,a,args):

    all_variants = pow(2,amount)
    res_dict = {}
    start_divider = 4
    var_count = int(all_variants/start_divider)
    for n in range(amount-1):
        res_list = []
        cycle_count = int(all_variants/var_count)
        
        for k in range(cycle_count):
            if k % 2 == 0 :
                res_list += [True for num in range(var_count)]
            else:
                res_list += [False for num in range(var_count)]
        res_dict[args[n+1]] = res_list
        var_count = int(var_count/2)
        
    return res_dict







def bool_to_int(answers_dict):
    new_dict = {}
    for letter in answers_dict:
        new_list  = [int(value) for value in answers_dict[letter]]
        new_dict[letter] = new_list
    return new_dict

def replace_letters(input_list,letter,number):
    for i in range(len(input_list)):
        if isinstance(input_list[i], list):
            replace_letters(input_list[i],letter,number)
        elif isinstance(input_list[i],str) and input_list[i] == (f'-{letter}'):
            negated = int(negate(number))
            input_list[i] = negated
        elif isinstance(input_list[i], str) and input_list[i] == letter:
            input_list[i] = number
# Display the DataFrame
def data_create(args):
    amount = len(args)
    all_variants = pow(2,amount)
    amount_pow_half = int(all_variants/2)
    a = [True] * amount_pow_half + [False] * amount_pow_half

    result_dict = {args[0]:a}
    generated = generate_next_list(amount,a,args)
    result_dict.update(generated)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    result_dict_new = bool_to_int(result_dict)

    df1 = pd.DataFrame(result_dict_new)
    print(df1)
    
    return result_dict_new
    
def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def solve(expr_list):
    
    
    if len(expr_list) >1:
        
        
        if type(expr_list[0])==str and '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if type(expr_list[2])==str and'-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
    
        for index,item in enumerate(expr_list):
            if type(item) == str and item.isdigit():
                expr_list[index] = int(item)
        match expr_list[1]:

            case 'or':
                answer = log_or(expr_list[0],expr_list[2])
                
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
               
            case 'imp':
                answer = implication(expr_list[0],expr_list[2])
               
            case 'eq':
                answer = log_equal(expr_list[0],expr_list[2])
                
        
        expr_list = answer
        
        return expr_list

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        for i,item in enumerate(nested_list):
            check_var = process_nested_lists(item)
            if check_var is not None:
                nested_list[i] = process_nested_lists(item)
        return solve(nested_list)



def solve_for_all(problem,result_dict):
    max_length = max(len(lst) for lst in result_dict.values())
    keys = result_dict.keys()
    print(keys,'keys')
    results_list = []
    for i in range(max_length):
        values = []
        

        for key, value_list in result_dict.items():
            if i < len(value_list):
                print(value_list[i],"value list curr value")
                values.append(value_list[i])
         
        
        nested_list = nestedExpr().parseString(problem).asList()
        nested_list = nested_list[0]
        print(nested_list,'iyo')
        for i,key in enumerate(keys):
            replace_letters(nested_list,key,values[i])
            print(key,values[i], 'eseni gadaeca')
        print(nested_list,'gaxda')
        
        processed = process_nested_lists(nested_list)
        
        
        results_list.append(processed)
    return results_list
        


def get_text_textfield(textfield):
    return textfield.get("1.0", "end-1c")



def display_dataframe(df):
    text_widget.delete(1.0, tk.END) 
    text_widget.insert(tk.END, df)

def solve_text():
    problem = f'({get_text_textfield(text_field)})'
    args = get_text_textfield(text_field_args).split(',')
    print(problem)
    print(args)
    result_dict_new = data_create(args)
    solve_for_all(problem,result_dict_new)
    result_dict_new['res'] = solve_for_all(problem,result_dict_new)
    print(result_dict_new)

    df2 = pd.DataFrame(result_dict_new)
    display_dataframe(df2)
    print(df2)

def on_button_click():
    label.config(text="Button Clicked!")

def display_text():
    text_content = text_field.get("1.0", "end-1c")
    print("Text entered:", text_content)
    


    

root = tk.Tk()
root.title("calculator")
root.geometry('600x500')



label = tk.Label(root, text="Enter Text:")
label.pack()

text_field = tk.Text(root, height=2, width=40) 
text_field.pack()

add_arguments_label = tk.Label(root,text="add arguments:(p,q,r...)")
add_arguments_label.pack(pady=(10,0))

text_field_args = tk.Text(root, height=1, width=20)  
text_field_args.pack()


text_widget = scrolledtext.ScrolledText(root, width=40, height=10)
text_widget.pack(pady=20)

button = tk.Button(root, text="solve", command=solve_text)
button.pack()

info_label = tk.Label(root,text='type expression in the first box')
info_label.pack()
info_label_2 = tk.Label(root,text='use and,or,imp(implication),-(in front of letter),eq(equals)')
info_label_2.pack()
info_label_3 = tk.Label(root,text='then type values in second box(if you have p,q and r just type p,q,r and that\'s it)')
info_label_3.pack()
info_label_4 = tk.Label(root,text='third box is for results')
info_label_4.pack()

root.mainloop()


In [102]:
def is_one_dimensional(input_list):
    for item in input_list:
        if isinstance(item, list):
            return False
    return True

def solve(expr_list):
    
    
    if len(expr_list) >1:
        print(expr_list)
        
        if type(expr_list[0])==str and '-' in  expr_list[0]:
            expr_list[0] = negate(expr_list[0])
        if type(expr_list[2])==str and'-' in expr_list[2]:
            expr_list[2] = negate(expr_list[2])
    
        for index,item in enumerate(expr_list):
            if type(item) == str and item.isdigit():
                expr_list[index] = int(item)
        match expr_list[1]:

            case 'or':
                answer = log_or(expr_list[0],expr_list[2])
                print('or')
            case 'and':
                answer = log_and(expr_list[0],expr_list[2])
                print('and')
            case 'imp':
                answer = implication(expr_list[0],expr_list[2])
                print('imp')
            case 'equal':
                answer = log_equal(expr_list[0],expr_list[2])
                print('equal')
        
        expr_list = answer
        print(f"answer is {expr_list} ")
        return expr_list

def process_nested_lists(nested_list):
    if isinstance(nested_list, list):
        print(nested_list,' process')
        for i,item in enumerate(nested_list):
            check_var = process_nested_lists(item)
            if check_var is not None:
                nested_list[i] = process_nested_lists(item)
        return solve(nested_list)


# Example usage

nested_list = [['0', 'or', '0'], 'imp', [['1', 'and', '1'], 'and', ['1', 'or', '0']]]
process_nested_lists(nested_list)

[['0', 'or', '0'], 'imp', [['1', 'and', '1'], 'and', ['1', 'or', '0']]]  process
['0', 'or', '0']  process
['0', 'or', '0']
or
answer is 0 
[0, 'or', 0]  process
[0, 'or', 0]
or
answer is 0 
[['1', 'and', '1'], 'and', ['1', 'or', '0']]  process
['1', 'and', '1']  process
['1', 'and', '1']
and
answer is 1 
[1, 'and', 1]  process
[1, 'and', 1]
and
answer is 1 
['1', 'or', '0']  process
['1', 'or', '0']
or
answer is 1 
[1, 'or', 0]  process
[1, 'or', 0]
or
answer is 1 
[1, 'and', 1]
and
answer is 1 
[1, 'and', 1]  process
[1, 'and', 1]
and
answer is 1 
[0, 'imp', 1]
imp
answer is True 


True

In [2]:
import pandas as pd

def generate_next_list(amount,a,args):
    res_dict = {}
    all_variants = pow(2,amount)
    start_divider = 4
    var_count = int(all_variants/start_divider)
    for n in range(amount-1):
        res_list = []
        cycle_count = int(all_variants/var_count)
        
        for k in range(cycle_count):
            if k % 2 == 0 :
                res_list += [True for num in range(var_count)]
            else:
                res_list += [False for num in range(var_count)]
        res_dict[args[n+1]] = res_list
        var_count = int(var_count/2)
        
    return res_dict

args = ['p','q','r','t',"k",'l']
amount = len(args)
amount_pow = pow(2,amount)
amount_pow_half = int(amount_pow/2)
a = [True] * amount_pow_half + [False] * amount_pow_half


def bool_to_int(answers_dict):
    new_dict = {}
    for letter in answers_dict:
        new_list  = [int(value) for value in answers_dict[letter]]
        new_dict[letter] = new_list
    return new_dict
# Display the DataFrame

result_dict = {args[0]:a}
generated = generate_next_list(amount,a,args)
result_dict.update(generated)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
result_dict_new = bool_to_int(result_dict)

df1 = pd.DataFrame(result_dict_new)
print(df1)

    p  q  r  t  k  l
0   1  1  1  1  1  1
1   1  1  1  1  1  0
2   1  1  1  1  0  1
3   1  1  1  1  0  0
4   1  1  1  0  1  1
5   1  1  1  0  1  0
6   1  1  1  0  0  1
7   1  1  1  0  0  0
8   1  1  0  1  1  1
9   1  1  0  1  1  0
10  1  1  0  1  0  1
11  1  1  0  1  0  0
12  1  1  0  0  1  1
13  1  1  0  0  1  0
14  1  1  0  0  0  1
15  1  1  0  0  0  0
16  1  0  1  1  1  1
17  1  0  1  1  1  0
18  1  0  1  1  0  1
19  1  0  1  1  0  0
20  1  0  1  0  1  1
21  1  0  1  0  1  0
22  1  0  1  0  0  1
23  1  0  1  0  0  0
24  1  0  0  1  1  1
25  1  0  0  1  1  0
26  1  0  0  1  0  1
27  1  0  0  1  0  0
28  1  0  0  0  1  1
29  1  0  0  0  1  0
30  1  0  0  0  0  1
31  1  0  0  0  0  0
32  0  1  1  1  1  1
33  0  1  1  1  1  0
34  0  1  1  1  0  1
35  0  1  1  1  0  0
36  0  1  1  0  1  1
37  0  1  1  0  1  0
38  0  1  1  0  0  1
39  0  1  1  0  0  0
40  0  1  0  1  1  1
41  0  1  0  1  1  0
42  0  1  0  1  0  1
43  0  1  0  1  0  0
44  0  1  0  0  1  1
45  0  1  0  0  1  0
46  0  1  0  

In [21]:
import tkinter as tk

def get_text_textfield(textfield):
    return textfield.get("1.0", "end-1c")

def solve_text():
    problem = f'({get_text_textfield(text_field)})'
    args = get_text_textfield(text_field_args)
    print(problem)
    print(args)
    result_dict_new = data_create(args)
    solve_for_all(problem,result_dict_new)
    result_dict_new['res'] = solve_for_all(problem,result_dict_new)
    print(result_dict_new)

    df2 = pd.DataFrame(result_dict_new)
    print(df2)

def on_button_click():
    label.config(text="Button Clicked!")

def display_text():
    text_content = text_field.get("1.0", "end-1c")
    print("Text entered:", text_content)
    

    

root = tk.Tk()
root.title("calculator")
root.geometry('800x600')

label = tk.Label(root, text="Hello, GUI!")
label.pack()
# Create a label
label = tk.Label(root, text="Enter Text:")
label.pack()

text_field = tk.Text(root, height=2, width=40) 
text_field.pack()

add_arguments_label = tk.Label(root,text="add arguments:(p,q,r...)")
add_arguments_label.pack(pady=(10,0))

text_field_args = tk.Text(root, height=1, width=20)  
text_field_args.pack()

button = tk.Button(root, text="solve", command=solve_text)
button.pack()

root.mainloop()
